In [ ]:
# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [ ]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [ ]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [ ]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [ ]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [ ]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [ ]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)

In [ ]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.initializers import glorot_normal, random_normal
from keras.optimizers import Adam

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

- Defining the Architecture of LSTM

In [16]:
# Initiliazing the sequential model
# model = Sequential()
# # Configuring the parameters
# model.add(LSTM(n_hidden,return_sequences=True, input_shape=(timesteps, input_dim),kernel_initializer = "glorot_normal"))
# model.add(Dropout(0.5))
# model.add(LSTM(32, return_sequences=False))
# # Adding a dropout layer
# model.add(Dropout(0.5))
# # Adding a dense output layer with sigmoid activation
# model.add(Dense(n_classes, activation='sigmoid'))
# model.summary()

In [17]:
# # Compiling the model
# model.compile(loss='categorical_crossentropy',
#               optimizer='Adam',
#               metrics=['accuracy'])

In [ ]:
def create_network(dropout_rate=0.5, init = "random_normal", neurons=32):
    epochs = 30
    batch_size = 32
    n_hidden = 32
    # Start neural network
    model = Sequential()
    # Configuring the parameters
    model.add(LSTM(neurons,return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer = init))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(neurons, return_sequences=False))
    model.add(Dropout(dropout_rate))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))
    
    optimizer = Adam(lr = learning_rate)
    # Compile neural network
    model.compile(loss='categorical_crossentropy',
              optimizer= optimizer,
              metrics=['accuracy']) # Accuracy performance metric
    

    return model

In [ ]:
epochs = 30
batch_size = 32
n_hidden = 32

In [ ]:
model = KerasClassifier(build_fn=create_network, verbose=1, batch_size = batch_size, epochs = epochs)

In [ ]:
# init = ["random_normal", "glorot_normal"]
# # learn_rate = [0.0001, 0.001, 0.01, 0.1]
# dropout_rate = [0.5, 0.6, 0.7, 0.8, 0.9]
# neurons = [32, 64]
# hyperparameters = dict(init=init, dropout_rate = dropout_rate, neurons = neurons)
# grid = GridSearchCV(estimator=model, param_grid=hyperparameters, n_jobs=-1, cv=2, verbose=1)
# grid_result = grid.fit(X_train, Y_train)


In [ ]:
# grid_result.best_params_

In [28]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import BatchNormalization

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
# epochs = 30
# batch_size = 32
# n_hidden = 32
# Start neural network
model = Sequential()
# Configuring the parameters
model.add(LSTM(32,return_sequences=True, input_shape=(timesteps, input_dim), kernel_initializer = "glorot_normal"))
# model.add(Dropout(0.4))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))

# optimizer = Adam(lr = learning_rate)
# Compile neural network
model.compile(loss='categorical_crossentropy',
          optimizer= "Adam",
          metrics=['accuracy']) # Accuracy performance metric

In [ ]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),
          epochs=25, callbacks=[learning_rate_reduction])

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [ ]:
score = model.evaluate(X_test, Y_test)

In [ ]:
score

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [58]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(128,return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(LSTM(128))
model.add(Dropout(0.25))

# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))

# optimizer = Adam(lr = learning_rate)
# Compile neural network
model.summary()
model.compile(loss='categorical_crossentropy',
          optimizer= "rmsprop",
          metrics=['accuracy']) # Accuracy performance metric


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128, 128)          70656     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128, 128)          512       
_________________________________________________________________
dropout_8 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 774       
Total params: 203,526
Trainable params: 203,270
Non-trainable params: 256
_________________________________________________________________


In [79]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=500,
          validation_data=(X_test, Y_test),
          epochs=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/1
7352/7352 [==============================] - 53s 7ms/step - loss: 0.0358 - acc: 0.9839 - val_loss: 0.2412 - val_acc: 0.9464


In [ ]:
# model.save('har.hdf5')

# from keras.models import load_model

# model = load_model('har.hdf5')

In [80]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                 20      393        73        0                   0   
STANDING                 0       25       504        1                   0   
WALKING                  0        3         0      489                   4   
WALKING_DOWNSTAIRS       0        2         0        0                 416   
WALKING_UPSTAIRS         0        4         0        1                  16   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            5  
STANDING                           2  
WALKING                            0  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 450  


In [81]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 8s 3ms/step


[0.24118039630331928, 0.9463861554122837]

In [40]:
1

1

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning